In [11]:
# !pip install deap

In [1]:
import pandas as pd

# Load the Excel file
file_path = r'C:\Users\Ibrahim Patwary\Desktop\Data augmentation\breast_cancer_updated_dataset.xlsx'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path)


In [2]:
df.shape

(2540, 10)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Class               2540 non-null   object 
 1   SIFT_keypoints      2540 non-null   int64  
 2   Extreme_points_sum  2540 non-null   int64  
 3   WH_ratio            2540 non-null   float64
 4   Elliptic_ratio      2540 non-null   float64
 5   Circularity         2540 non-null   float64
 6   Harris_corners      2540 non-null   int64  
 7   Convex_hull_ratio   2540 non-null   float64
 8   Brightness          2540 non-null   float64
 9   Width_height_ratio  2540 non-null   float64
dtypes: float64(6), int64(3), object(1)
memory usage: 198.6+ KB


In [4]:
df.columns

Index(['Class', 'SIFT_keypoints', 'Extreme_points_sum', 'WH_ratio',
       'Elliptic_ratio', 'Circularity', 'Harris_corners', 'Convex_hull_ratio',
       'Brightness', 'Width_height_ratio'],
      dtype='object')

In [10]:
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import warnings

# Suppress specific warnings (if you want)
warnings.filterwarnings("ignore")

# Load the Excel file
file_path = r'C:\Users\Ibrahim Patwary\Desktop\Data augmentation\breast_cancer_updated_dataset.xlsx'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path)

# Prepare the data
X = df[['SIFT_keypoints', 'Extreme_points_sum', 'WH_ratio', 'Elliptic_ratio', 
        'Circularity', 'Harris_corners', 'Convex_hull_ratio', 'Brightness', 
        'Width_height_ratio']]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

y = df['Class']  # Assuming you've already encoded the Class column

# Encode the class labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define the objective functions
def classification_accuracy(X, y, feature_mask):
    # Use NumPy array indexing instead of .iloc
    selected_features = X[:, feature_mask]
    if selected_features.shape[1] == 0:
        return 0  # Return 0 if no features are selected
    clf = SVC()
    scores = cross_val_score(clf, selected_features, y, cv=5)
    return scores.mean()

def feature_count(feature_mask):
    return sum(feature_mask)

def evaluate(individual, X, y):
    feature_mask = [bool(gene) for gene in individual]
    accuracy = classification_accuracy(X, y, feature_mask)
    count = feature_count(feature_mask)
    return accuracy, -count  # We negate count because we want to minimize it

# Create fitness and individual classes
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0))  # Maximize both objectives
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Initialize the toolbox
toolbox = base.Toolbox()

# Define gene and individual creation
toolbox.register("attr_bool", np.random.choice, [True, False])
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=9)  # 9 features
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define evaluation function
toolbox.register("evaluate", evaluate, X=X_scaled, y=y_encoded)

# Define genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selNSGA2)

# Run the algorithm
def main():
    pop = toolbox.population(n=100)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=True)
    return pop

if __name__ == "__main__":
    final_pop = main()
    
    # Extract and print the best solutions
    best_solutions = tools.sortNondominated(final_pop, len(final_pop), first_front_only=True)[0]
    for idx, solution in enumerate(best_solutions):
        print(f"Solution {idx + 1}:")
        print(f"Selected features: {[i for i, gene in enumerate(solution) if gene]}")
        print(f"Accuracy: {solution.fitness.values[0]:.4f}")
        print(f"Number of features: {-solution.fitness.values[1]}")
        print()


gen	nevals
0  	100   
1  	70    
2  	65    
3  	59    
4  	71    
5  	62    
6  	64    
7  	53    
8  	55    
9  	50    
10 	51    
11 	55    
12 	67    
13 	67    
14 	64    
15 	74    
16 	57    
17 	65    
18 	64    
19 	56    
20 	59    
21 	59    
22 	53    
23 	59    
24 	65    
25 	68    
26 	62    
27 	60    
28 	57    
29 	62    
30 	56    
31 	69    
32 	49    
33 	57    
34 	48    
35 	66    
36 	60    
37 	63    
38 	67    
39 	60    
40 	55    
41 	64    
42 	49    
43 	55    
44 	70    
45 	66    
46 	59    
47 	59    
48 	49    
49 	62    
50 	57    
Solution 1:
Selected features: [0, 3, 6]
Accuracy: 0.8902
Number of features: 3.0

Solution 2:
Selected features: [0, 3, 6]
Accuracy: 0.8902
Number of features: 3.0

Solution 3:
Selected features: [0, 3, 6]
Accuracy: 0.8902
Number of features: 3.0

Solution 4:
Selected features: [0, 3, 6]
Accuracy: 0.8902
Number of features: 3.0

Solution 5:
Selected features: [0, 3, 6]
Accuracy: 0.8902
Number of features: 3.0

Solution 6:
S